In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install boto3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 9.8 MB/s eta 0:00:00


In [ ]:
import boto3
import pandas as pd

In [ ]:
import getpass

aws_key_id = getpass.getpass('Enter your AWS access key ID:')
aws_secret_access_key = getpass.getpass('Enter your AWS secret access key:')

import os

os.environ['AWS_ACCESS_KEY_ID'] = aws_key_id
os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_access_key


Enter your AWS access key ID:··········
Enter your AWS secret access key:··········


In [ ]:

bucket_name = 'miniproject90'
s3 = boto3.resource('s3', region_name='us-east-1')
bucket = s3.Bucket(bucket_name)



In [ ]:
for obj in bucket.objects.all():
    print(obj.key)


Transcriptions_all1000.csv
recepie-test_02May2024_1714670093166/recepie-test_02May2024_1714670093166_part00000.csv


In [ ]:
obj_key = 'recepie-test_02May2024_1714670093166/recepie-test_02May2024_1714670093166_part00000.csv'
obj = bucket.Object(obj_key)

In [ ]:
body = obj.get()['Body']
df = pd.read_csv(body)

In [ ]:
df

,Num,Channel,ChannelList,VideosDownload,MP3File,Transcripts
0,1,TEDEd,https://www.youtube.com/watch?v=D9N7QaIOkG8&li...,How does heart transplant surgery work - Roni ...,How does heart transplant surgery work - Roni ...,What is a heart Your heart beats more than 10...
1,2,TEDEd,https://www.youtube.com/watch?v=D9N7QaIOkG8&li...,The myth of Hades and Persephone - Iseult Gill...,The myth of Hades and Persephone - Iseult Gill...,Every year before the ancient Greeks sowed the...
2,3,TEDEd,https://www.youtube.com/watch?v=D9N7QaIOkG8&li...,The tragic myth of the Sun Gods son - Iseult G...,The tragic myth of the Sun Gods son - Iseult G...,Every morning Helios harnessed his winged hors...
3,4,TEDEd,https://www.youtube.com/watch?v=D9N7QaIOkG8&li...,What is the rarest color in nature - Victoria ...,What is the rarest color in nature - Victoria ...,Every color you see in front of you can be fou...
4,5,TEDEd,https://www.youtube.com/watch?v=D9N7QaIOkG8&li...,Why a sausage can do what your gloves cannot -...,Why a sausage can do what your gloves cannot -...,In 2010 South Korea experienced a particularly...
...,...,...,...,...,...,...
994,995,Zach Star,https://www.youtube.com/watch?v=5EYFlnI_0ew&li...,The Physics Major (Part 2),NaN,Moving on from the last video the next class i...
995,996,Zach Star,https://www.youtube.com/watch?v=5EYFlnI_0ew&li...,The History of Physics and Its Applications,NaN,Thousands of years of physics research and dis...
996,997,Zach Star,https://www.youtube.com/watch?v=5EYFlnI_0ew&li...,The History of Physics (Part 2),NaN,Like I said in the last video we are about to ...
997,998,Zach Star,https://www.youtube.com/watch?v=5EYFlnI_0ew&li...,Physics Vs Engineering Which Is Best For You,NaN,Physics and engineering are two very common to...


Models....

In [ ]:
!pip install gensim
!pip install nltk


In [ ]:
!pip install transformers
!pip install bert-extractive-summarizer

In [ ]:
!pip install spacy

In [ ]:
from summarizer import Summarizer,TransformerSummarizer

In [ ]:
# ##works for one row

# bert_model = Summarizer()
# bert_summary = ''.join(bert_model(df['Transcripts'][0], min_length=60))
# print(bert_summary)


IndentationError: unexpected indent (<ipython-input-40-cfae88bbface>, line 3)

Split the dataset: 80 - 20

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [ ]:
print(len(train_df))
print(len(test_df))

799
200


BERT Model

In [ ]:
from transformers import pipeline

# Initialize the summarization pipeline
summarizer = pipeline("summarization")

# Function to generate summaries for a given text
def generate_summary(text, min_length=50, max_input_length=500):
    try:
        # Generate summary for a portion of the text
        summary = summarizer(text[:max_input_length], min_length=min_length, max_length=len(text) // 2)
        return summary[0]['summary_text']

    except Exception as e:
        print(f"Error generating summary: {e}")
        return ""

# Apply the summary generation function to the first 10 rows of your dataframes
train_df_first_10 = train_df.head(10)
test_df_first_10 = test_df.head(10)

train_df_first_10['SummariesBERT'] = train_df_first_10['Transcripts'].apply(generate_summary)
test_df_first_10['SummariesBERT'] = test_df_first_10['Transcripts'].apply(generate_summary)

# Print summaries for verification
for i, row in train_df_first_10.iterrows():
    print(f"Summary for training sample {i}: {row['SummariesBERT']}")

for i, row in test_df_first_10.iterrows():
    print(f"Summary for testing sample {i}: {row['SummariesBERT']}")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Your max_length is set to 2484, but your input_length is only 102. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)
Your max_length is set to 7587, but your input_length is only 95. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)
Your max_length is set to 1732, but your input_length is only 95. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)
Your max_l

Summary for training sample 778:  In the last lecture we talked about strings We discussed how we can split strings how strings work and so on And again there are several functions that we can use with strings and well get to them later in the course but basically I just wanted to give you a quick introduction .
Summary for training sample 286:  Part eight of Python for Finance video tutorial series . We are going to search the entire market so that we can create the most optimized portfolios to maximize return while minimizing your overall risk . Im going to talk again more about the Sharpe ratio .
Summary for training sample 165:  Always before we kept solving equations and like 2x plus 1 equals 0 we needed to move away from whole numbers . We had coefficients that were whole numbers but the solutions werent whole numbers anymore . But with complex numbers you dont have to . move away anymore thats the end of the story .
Summary for training sample 960:  Imagine taking a notebook a p

<ipython-input-44-96c2dfc853f5>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_first_10['SummariesBERT'] = test_df_first_10['Transcripts'].apply(generate_summary)


In [ ]:
import pandas as pd

# Function to generate reference summary for a single row
def generate_reference_summary(text):
    # Extract the first few sentences as the reference summary
    sentences = text.split('.')
    reference_summary = '.'.join(sentences[:2])  # Extract the first two sentences
    return reference_summary

# Apply the reference summary generation function to the first 10 rows of your dataframe
train_df_first_10['ReferenceSummary'] = train_df_first_10['Transcripts'].apply(generate_reference_summary)
test_df_first_10['ReferenceSummary'] = test_df_first_10['Transcripts'].apply(generate_reference_summary)

# Print reference summaries for verification
for i, row in train_df_first_10.iterrows():
    print(f"Reference summary for training sample {i}: {row['ReferenceSummary']}")

for i, row in test_df_first_10.iterrows():
    print(f"Reference summary for testing sample {i}: {row['ReferenceSummary']}")

Reference summary for training sample 778: Alright so in the last lecture we talked about strings We discussed how we can split strings slice strings how strings work and so on And again there are several functions that we can use with strings and well get to them later in the course but basically I just wanted to give you a quick introduction on how swing strings work Now we get on to lists So for example lets say I had to go to maybe Fred Meyer or Target or any shopping store thats maybe near you and I was creating a list So lets create a shopping list of the things I need to buy And Im going to be using a string here So I have a shopping list maybe I need eggs eggs I might need some carrots oh I need milk cherries and lets add one more apples So lets say I had to go to a shopping store and I had all these different items that I had to buy So right now what the problem is is that if I need to know whats the first item in my list what do I have remaining Stuff like that I cant do that

<ipython-input-46-62975bced893>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_first_10['ReferenceSummary'] = train_df_first_10['Transcripts'].apply(generate_reference_summary)
<ipython-input-46-62975bced893>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_first_10['ReferenceSummary'] = test_df_first_10['Transcripts'].apply(generate_reference_summary)


In [ ]:
from transformers import BartTokenizer

# Define the directory path
directory = '/content/drive/My Drive/Pretrained Model'

# Create the directory if it doesn't exist
os.makedirs(directory, exist_ok=True)

# Save the BERT summarization model
torch.save(summarizer.model.state_dict(), os.path.join(directory, 'bert_summarizer_model.pth'))

# Use the BartTokenizer class to save the tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
tokenizer.save_pretrained(directory)


('/content/drive/My Drive/Pretrained Model/tokenizer_config.json',
 '/content/drive/My Drive/Pretrained Model/special_tokens_map.json',
 '/content/drive/My Drive/Pretrained Model/vocab.json',
 '/content/drive/My Drive/Pretrained Model/merges.txt',
 '/content/drive/My Drive/Pretrained Model/added_tokens.json')

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
train_df_first_10
test_df_first_10

,Num,Channel,ChannelList,VideosDownload,MP3File,Transcripts,SummariesBERT,ReferenceSummary
453,454,English Easy Practice,https://www.youtube.com/watch?v=yv77OZ_og-o&li...,English Story For Listening Practice English ...,English Story For Listening Practice English ...,Hello welcome to the English Easy Practice Cou...,English Easy Practice Course is an easy way t...,Hello welcome to the English Easy Practice Cou...
793,794,NaN,NaN,NaN,Should You Buy Vue JS 2 The Complete Guide .mp3,Hey guys today were gonna look at the Vuejs 2 ...,The Vuejs 2 complete guide is a complete guid...,Hey guys today were gonna look at the Vuejs 2 ...
209,210,Edureka,https://www.youtube.com/watch?v=xuB1Id2Wxak&li...,Git Merge Conflict Tutorial Resolving Merge C...,Git Merge Conflict Tutorial Resolving Merge Co...,Hello everyone this is Reshma from Edureka and...,Reshma from Edureka gives a brief introductio...,Hello everyone this is Reshma from Edureka and...
309,310,Freethink,https://www.youtube.com/watch?v=J4SaSfnHK3I&li...,Vertical farms could take over the world Hard...,Vertical farms could take over the world Hard...,This is the first farm on Mars Okay technicall...,Vertical farming could be the future of farmi...,This is the first farm on Mars Okay technicall...
740,741,NaN,NaN,NaN,Transcription for 9 Arithmetic Operators in Ja...,Welcome back Aliens My name is Abin Reddy and...,Abin Reddy continues series on JavaScript . H...,Welcome back Aliens My name is Abin Reddy and...
578,579,StatQuest,https://www.youtube.com/watch?v=FgakZw6K1QQ&li...,StatQuest t-SNE Clearly Explained.mp4,StatQuest t-SNE Clearly Explained.mp3,Im drawing a graph Doesnt it look cool But I d...,StackQuest is brought to you by the friendly ...,Im drawing a graph Doesnt it look cool But I d...
895,896,Knowledgia,https://www.youtube.com/watch?v=4SzoEAOmLng&li...,Did the Romans explore deeper into Africa,NaN,The Roman Empire included large territorial ho...,The Roman Empire included large territorial h...,The Roman Empire included large territorial ho...
545,546,Edureka,https://www.youtube.com/watch?v=8PopR3x-VMY&li...,Understanding Structures in C Programming Edu...,Understanding Structures in C Programming Edu...,So as we have spoken earlier we need a way to ...,We need a way to represent complex data types...,So as we have spoken earlier we need a way to ...
436,437,Alice Keeler,https://www.youtube.com/watch?v=7gIJMplnmhk&li...,Part 3 You Died in Minecraft Education Edition...,Part 3 You Died in Minecraft Education Edition...,Well let this be a lesson to you to pay attent...,A zombie came and killed me while I was makin...,Well let this be a lesson to you to pay attent...
678,679,StatQuest,https://www.youtube.com/watch?v=FgakZw6K1QQ&li...,Ken Jees 66DaysOfData Challenge Clearly Explai...,Ken Jees 66DaysOfData Challenge Clearly Explai...,66 days of data thats what Im gonna talk about...,Josh Starmor will talk about Kenjis 66 days o...,66 days of data thats what Im gonna talk about...


In [ ]:
!pip install rouge

In [ ]:
import sys
from rouge import Rouge

# Increase the recursion limit
sys.setrecursionlimit(10**6)

# Initialize the Rouge object
rouge = Rouge()

# Iterate over the first 10 rows of the DataFrame
for i, row in test_df_first_10.iterrows():
    reference_summary = row['ReferenceSummary']
    generated_summary = row['SummariesBERT']
    scores = rouge.get_scores(generated_summary, reference_summary)[0]
    test_df_first_10.at[i, 'ROUGE-1'] = scores['rouge-1']['f']
    test_df_first_10.at[i, 'ROUGE-2'] = scores['rouge-2']['f']
    test_df_first_10.at[i, 'ROUGE-L'] = scores['rouge-l']['f']



# Print the dataframe with the ROUGE scores
print(test_df_first_10)



<ipython-input-51-a6369d9dba89>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_first_10.at[i, 'ROUGE-1'] = scores['rouge-1']['f']
<ipython-input-51-a6369d9dba89>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_first_10.at[i, 'ROUGE-2'] = scores['rouge-2']['f']
<ipython-input-51-a6369d9dba89>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

     Num                Channel  \
453  454  English Easy Practice   
793  794                    NaN   
209  210                Edureka   
309  310              Freethink   
740  741                    NaN   
578  579              StatQuest   
895  896             Knowledgia   
545  546                Edureka   
436  437           Alice Keeler   
678  679              StatQuest   

                                           ChannelList  \
453  https://www.youtube.com/watch?v=yv77OZ_og-o&li...   
793                                                NaN   
209  https://www.youtube.com/watch?v=xuB1Id2Wxak&li...   
309  https://www.youtube.com/watch?v=J4SaSfnHK3I&li...   
740                                                NaN   
578  https://www.youtube.com/watch?v=FgakZw6K1QQ&li...   
895  https://www.youtube.com/watch?v=4SzoEAOmLng&li...   
545  https://www.youtube.com/watch?v=8PopR3x-VMY&li...   
436  https://www.youtube.com/watch?v=7gIJMplnmhk&li...   
678  https://www.youtube.com/watch

In [ ]:
test_df_first_10.to_csv('test_dfWithRogueScore.csv',index=False)

In [ ]:


# # Create a new column for the summaries
# df['Summaries'] = ""

# # Initialize the BERT summarizer
# bert_model = Summarizer()

# # Loop through the rows of the dataframe and generate summaries
# for i, row in df.iterrows():
#     summary = bert_model(row['Transcripts'], min_length=60)
#     df.at[i, 'Summaries'] = summary

# # Print the dataframe with the summaries
# print(df)


Saving the model in drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/Pretrained Model'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
torch.save(bert_model, '/content/drive/My Drive/Pretrained Model/bert_summarizer_model.pth')

NameError: name 'bert_model' is not defined

GPT2 Model

In [ ]:
# ### Works for one row
# GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
# full = ''.join(GPT2_model(df['Transcripts'][0], min_length=60))
# print(full)

In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from concurrent.futures import ThreadPoolExecutor

# Initialize the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Function to generate summary for a single row using GPT-2
def generate_summary(text):
    try:
        inputs = tokenizer.encode("summarize: " + text[:500], return_tensors="pt", max_length=1024, truncation=True)
        outputs = model.generate(inputs, max_length=150, min_length=50, num_beams=5, early_stopping=True)
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return summary
    except Exception as e:
        print(f"Error generating summary: {e}")
        return ""

# Function to process summaries for multiple rows in parallel
def process_rows(df):
    summaries = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(generate_summary, row['Transcripts']) for _, row in df.iterrows()]
        for future in futures:
            summaries.append(future.result())
    return summaries

# Apply the summary generation function to the first 10 rows of your dataframes
train_df_first_10 = train_df.head(10)
test_df_first_10 = test_df.head(10)

# Process summaries for the first 10 rows in parallel
train_summaries = process_rows(train_df_first_10)
test_summaries = process_rows(test_df_first_10)

# Assign summaries to dataframe
train_df_first_10['SummariesGPT2'] = train_summaries
test_df_first_10['SummariesGPT2'] = test_summaries

# Print summaries for verification
for i, row in train_df_first_10.iterrows():
    print(f"Summary for training sample {i}: {row['SummariesGPT2']}")

for i, row in test_df_first_10.iterrows():
    print(f"Summary for testing sample {i}: {row['SummariesGPT2']}")

# Save the GPT-2 model
model.save_pretrained('/content/drive/My Drive/Pretrained Model/gpt2_summarizer_model/')
tokenizer.save_pretrained('/content/drive/My Drive/Pretrained Model/gpt2_summarizer_model/')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Summary for training sample 778: summarize: Alright so in the last lecture we talked about strings We discussed how we can split strings slice strings how strings work and so on And again there are several functions that we can use with strings and well get to them later in the course but basically I just wanted to give you a quick introduction on how swing strings work Now we get on to lists So for example lets say I had to go to maybe Fred Meyer or Target or any shopping store thats maybe near you and I was creating a list So lets creat a list of all the items that are in your shopping list and then we create a list of all the items that are in your shopping list and then we create a list of all the items that are in your shopping list and then
Summary for training sample 286: summarize: Well hello internet and welcome to part eight of my Python for Finance video tutorial series In the last video I showed you how to create efficient portfolios using the Markowitz portfolio optimizati

('/content/drive/My Drive/Pretrained Model/gpt2_summarizer_model/tokenizer_config.json',
 '/content/drive/My Drive/Pretrained Model/gpt2_summarizer_model/special_tokens_map.json',
 '/content/drive/My Drive/Pretrained Model/gpt2_summarizer_model/vocab.json',
 '/content/drive/My Drive/Pretrained Model/gpt2_summarizer_model/merges.txt',
 '/content/drive/My Drive/Pretrained Model/gpt2_summarizer_model/added_tokens.json')

In [ ]:
import pandas as pd

# Function to generate reference summary for a single row
def generate_reference_summary(text):
    # Extract the first few sentences as the reference summary
    sentences = text.split('.')
    reference_summary = '.'.join(sentences[:2])  # Extract the first two sentences
    return reference_summary

# Apply the reference summary generation function to the first 10 rows of your dataframe
train_df_first_10['ReferenceSummary'] = train_df_first_10['Transcripts'].apply(generate_reference_summary)
test_df_first_10['ReferenceSummary'] = test_df_first_10['Transcripts'].apply(generate_reference_summary)

# Print reference summaries for verification
for i, row in train_df_first_10.iterrows():
    print(f"Reference summary for training sample {i}: {row['ReferenceSummary']}")

for i, row in test_df_first_10.iterrows():
    print(f"Reference summary for testing sample {i}: {row['ReferenceSummary']}")

Reference summary for training sample 778: Alright so in the last lecture we talked about strings We discussed how we can split strings slice strings how strings work and so on And again there are several functions that we can use with strings and well get to them later in the course but basically I just wanted to give you a quick introduction on how swing strings work Now we get on to lists So for example lets say I had to go to maybe Fred Meyer or Target or any shopping store thats maybe near you and I was creating a list So lets create a shopping list of the things I need to buy And Im going to be using a string here So I have a shopping list maybe I need eggs eggs I might need some carrots oh I need milk cherries and lets add one more apples So lets say I had to go to a shopping store and I had all these different items that I had to buy So right now what the problem is is that if I need to know whats the first item in my list what do I have remaining Stuff like that I cant do that

<ipython-input-34-62975bced893>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_first_10['ReferenceSummary'] = train_df_first_10['Transcripts'].apply(generate_reference_summary)
<ipython-input-34-62975bced893>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_first_10['ReferenceSummary'] = test_df_first_10['Transcripts'].apply(generate_reference_summary)


In [ ]:
train_df_first_10
test_df_first_10

,Num,Channel,ChannelList,VideosDownload,MP3File,Transcripts,SummariesGPT2,ReferenceSummary
453,454,English Easy Practice,https://www.youtube.com/watch?v=yv77OZ_og-o&li...,English Story For Listening Practice English ...,English Story For Listening Practice English ...,Hello welcome to the English Easy Practice Cou...,summarize: Hello welcome to the English Easy P...,Hello welcome to the English Easy Practice Cou...
793,794,NaN,NaN,NaN,Should You Buy Vue JS 2 The Complete Guide .mp3,Hey guys today were gonna look at the Vuejs 2 ...,summarize: Hey guys today were gonna look at t...,Hey guys today were gonna look at the Vuejs 2 ...
209,210,Edureka,https://www.youtube.com/watch?v=xuB1Id2Wxak&li...,Git Merge Conflict Tutorial Resolving Merge C...,Git Merge Conflict Tutorial Resolving Merge Co...,Hello everyone this is Reshma from Edureka and...,summarize: Hello everyone this is Reshma from ...,Hello everyone this is Reshma from Edureka and...
309,310,Freethink,https://www.youtube.com/watch?v=J4SaSfnHK3I&li...,Vertical farms could take over the world Hard...,Vertical farms could take over the world Hard...,This is the first farm on Mars Okay technicall...,summarize: This is the first farm on Mars Okay...,This is the first farm on Mars Okay technicall...
740,741,NaN,NaN,NaN,Transcription for 9 Arithmetic Operators in Ja...,Welcome back Aliens My name is Abin Reddy and...,summarize: Welcome back Aliens My name is Abi...,Welcome back Aliens My name is Abin Reddy and...
578,579,StatQuest,https://www.youtube.com/watch?v=FgakZw6K1QQ&li...,StatQuest t-SNE Clearly Explained.mp4,StatQuest t-SNE Clearly Explained.mp3,Im drawing a graph Doesnt it look cool But I d...,summarize: Im drawing a graph Doesnt it look c...,Im drawing a graph Doesnt it look cool But I d...
895,896,Knowledgia,https://www.youtube.com/watch?v=4SzoEAOmLng&li...,Did the Romans explore deeper into Africa,NaN,The Roman Empire included large territorial ho...,summarize: The Roman Empire included large ter...,The Roman Empire included large territorial ho...
545,546,Edureka,https://www.youtube.com/watch?v=8PopR3x-VMY&li...,Understanding Structures in C Programming Edu...,Understanding Structures in C Programming Edu...,So as we have spoken earlier we need a way to ...,summarize: So as we have spoken earlier we nee...,So as we have spoken earlier we need a way to ...
436,437,Alice Keeler,https://www.youtube.com/watch?v=7gIJMplnmhk&li...,Part 3 You Died in Minecraft Education Edition...,Part 3 You Died in Minecraft Education Edition...,Well let this be a lesson to you to pay attent...,summarize: Well let this be a lesson to you to...,Well let this be a lesson to you to pay attent...
678,679,StatQuest,https://www.youtube.com/watch?v=FgakZw6K1QQ&li...,Ken Jees 66DaysOfData Challenge Clearly Explai...,Ken Jees 66DaysOfData Challenge Clearly Explai...,66 days of data thats what Im gonna talk about...,summarize: 66 days of data thats what Im gonna...,66 days of data thats what Im gonna talk about...


In [ ]:


# # Create a new column for the summaries
# df['SummariesGPT'] = ""

# # Initialize the BERT summarizer
# GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")

# # Loop through the rows of the dataframe and generate summaries
# for i, row in df.iterrows():
#     summary = GPT2_model(row['Transcripts'], min_length=60)
#     df.at[i, 'SummariesGPT'] = summary

# # Print the dataframe with the summaries
# print(df)


In [ ]:
import torch
torch.save(GPT2_model, '/content/drive/My Drive/Pretrained Model/gpt_summarizer_modelNew.pth')

NameError: name 'GPT2_model' is not defined

In [ ]:
from rouge import Rouge

# Initialize the ROUGE metric
rouge = Rouge()

# Calculate the ROUGE scores for each row in the test set
for i, row in test_df_first_10.iterrows():
    reference_summary = row['ReferenceSummary']
    generated_summary = row['SummariesGPT2']
    scores = rouge.get_scores(generated_summary, reference_summary)[0]
    test_df_first_10[i, 'ROUGE-1 GPT2'] = scores['rouge-1']['f']
    test_df_first_10[i, 'ROUGE-2 GPT2'] = scores['rouge-2']['f']
    test_df_first_10[i, 'ROUGE-L GPT2'] = scores['rouge-l']['f']



# Print the dataframe with the ROUGE scores
print(test_df_first_10)


<ipython-input-37-b474cbfe439f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_first_10[i, 'ROUGE-1 GPT2'] = scores['rouge-1']['f']
<ipython-input-37-b474cbfe439f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_first_10[i, 'ROUGE-2 GPT2'] = scores['rouge-2']['f']
<ipython-input-37-b474cbfe439f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

     Num                Channel  \
453  454  English Easy Practice   
793  794                    NaN   
209  210                Edureka   
309  310              Freethink   
740  741                    NaN   
578  579              StatQuest   
895  896             Knowledgia   
545  546                Edureka   
436  437           Alice Keeler   
678  679              StatQuest   

                                           ChannelList  \
453  https://www.youtube.com/watch?v=yv77OZ_og-o&li...   
793                                                NaN   
209  https://www.youtube.com/watch?v=xuB1Id2Wxak&li...   
309  https://www.youtube.com/watch?v=J4SaSfnHK3I&li...   
740                                                NaN   
578  https://www.youtube.com/watch?v=FgakZw6K1QQ&li...   
895  https://www.youtube.com/watch?v=4SzoEAOmLng&li...   
545  https://www.youtube.com/watch?v=8PopR3x-VMY&li...   
436  https://www.youtube.com/watch?v=7gIJMplnmhk&li...   
678  https://www.youtube.com/watch

<ipython-input-37-b474cbfe439f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_first_10[i, 'ROUGE-1 GPT2'] = scores['rouge-1']['f']
<ipython-input-37-b474cbfe439f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_first_10[i, 'ROUGE-2 GPT2'] = scores['rouge-2']['f']
<ipython-input-37-b474cbfe439f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [ ]:
test_df.to_csv('test_dfWithRogueScoreGPT.csv',index=False)

BERT vs GPT2 - Rogue Score

In [ ]:
from rouge import Rouge

# Initialize the ROUGE metric
rouge = Rouge()

# Calculate the ROUGE scores for each row in the test set
for i, row in test_df.iterrows():
    reference_summary = row['SummariesBERT']
    generated_summary = row['SummariesGPT2']
    scores = rouge.get_scores(generated_summary, reference_summary)[0]
    test_df.at[i, 'ROUGE-1 BERT$GPT2'] = scores['rouge-1']['f']
    test_df.at[i, 'ROUGE-2 BERT$GPT2'] = scores['rouge-2']['f']
    test_df.at[i, 'ROUGE-L BERT$GPT2'] = scores['rouge-l']['f']



# Print the dataframe with the ROUGE scores
print(test_df)


                   Channel                                            MP3File  \
453  English Easy Practice  English Story For Listening  Practice English ...   
793                    NaN   Should You Buy Vue JS 2 The Complete Guide .mp3    
209                Edureka  Git Merge Conflict Tutorial Resolving Merge Co...   
309              Freethink  Vertical farms could take over the world  Hard...   
740                    NaN  Transcription for 9 Arithmetic Operators in Ja...   
..                     ...                                                ...   
78             Derek Banas              Download Every Stock in the World.mp3   
29         Sebastian Lague  [Unity] Procedural Cave Generation (E03 Creati...   
277            AsapSCIENCE  Harvesting Water Out Of Thin Air  Shut It Off ...   
261        Sebastian Lague                     Unity Gradient Editor (33).mp3   
423                   SXSW  Joy Ride Cast & Crew with Stephanie Hsu & Ashl...   

                           

In [ ]:
test_df.to_csv('test_dfWithRogueScoreBert$GPT.csv',index=False)

GPT2 vs BERT - Rogue Score

In [ ]:
from rouge import Rouge

# Initialize the ROUGE metric
rouge = Rouge()

# Calculate the ROUGE scores for each row in the test set
for i, row in test_df.iterrows():
    reference_summary = row['SummariesGPT2']
    generated_summary = row['SummariesBERT']
    scores = rouge.get_scores(generated_summary, reference_summary)[0]
    test_df.at[i, 'ROUGE-1 GPT2$BERT'] = scores['rouge-1']['f']
    test_df.at[i, 'ROUGE-2 GPT2$BERT'] = scores['rouge-2']['f']
    test_df.at[i, 'ROUGE-L GPT2$BERT'] = scores['rouge-l']['f']



# Print the dataframe with the ROUGE scores
print(test_df)


                   Channel                                            MP3File  \
453  English Easy Practice  English Story For Listening  Practice English ...   
793                    NaN   Should You Buy Vue JS 2 The Complete Guide .mp3    
209                Edureka  Git Merge Conflict Tutorial Resolving Merge Co...   
309              Freethink  Vertical farms could take over the world  Hard...   
740                    NaN  Transcription for 9 Arithmetic Operators in Ja...   
..                     ...                                                ...   
78             Derek Banas              Download Every Stock in the World.mp3   
29         Sebastian Lague  [Unity] Procedural Cave Generation (E03 Creati...   
277            AsapSCIENCE  Harvesting Water Out Of Thin Air  Shut It Off ...   
261        Sebastian Lague                     Unity Gradient Editor (33).mp3   
423                   SXSW  Joy Ride Cast & Crew with Stephanie Hsu & Ashl...   

                           

In [ ]:
test_df.to_csv('test_dfWithRogueScoreGPT$BERT.csv',index=False)

In [ ]:
# !pip install sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.8 MB/s eta 0:00:00
